In [12]:
import os
import datetime
import rasterio
import numpy as np
import geopandas as gpd

from rasterio.transform import from_origin
from rasterio.mask import mask
from netCDF4 import Dataset, num2date

In [13]:
#definisikan tanggal data yang mau diambil hari ini
rewind_day=1
cycle = '12'
today = datetime.date.today()-datetime.timedelta(days=rewind_day)
print("ECMWF.0125."+today.strftime('%Y%m%d')+cycle+"00.PREC.nc")

if (cycle=='00') :
    n=4
else :
    n=0

path_modified = "../repository/pre-processing/result-row"
result_file_name = f"ECMWF_new_3d.0125.{today.strftime('%Y%m%d')}{cycle}00.PREC.nc"

# Gabungkan path dengan nama file
file_path = os.path.join(path_modified, result_file_name)

print(file_path)

# Read NetCDF File
data = Dataset(file_path)

#Check waktu data
time = data.variables['time'][:]
dates = num2date(time, data.variables['time'].units)
print((dates[11+n]).strftime('%Y-%m-%d %H:%M:%S WIB'))
print((dates[19+n]).strftime('%Y-%m-%d %H:%M:%S WIB'))

lon = np.array(data.variables["lon"][:])
lat = np.array(data.variables["lat"][:])
prec = np.array(data.variables["rain"][:])

ECMWF.0125.202501291200.PREC.nc
../repository/pre-processing/result-row\ECMWF_new_3d.0125.202501291200.PREC.nc
2025-01-31 04:00:00 WIB
2025-02-01 04:00:00 WIB


In [14]:
# Inisialisasi akumulasi, meshgrid, tanggal
xx, yy = np.meshgrid(lon, lat)
accumulation = 0
start_date = dates[11+n - 1].strftime('%m%d%Y')

input_hour_dir = f"../repository/post-processing/nc_to_tiff/pch_hour_{start_date}/"
input_day_dir = f"../repository/post-processing/nc_to_tiff/pch_day_{start_date}/"
output_masked_hour_dir = f"../repository/post-processing/wms/hour/pch_hour_{start_date}/"
output_masked_day_dir = f"../repository/post-processing/wms/day/pch_day_{start_date}/"
mask_pulau = "../data/geojson/pulau.geojson"

os.makedirs(input_hour_dir, exist_ok=True)
os.makedirs(input_day_dir, exist_ok=True)
os.makedirs(output_masked_hour_dir, exist_ok=True)
os.makedirs(output_masked_day_dir, exist_ok=True)

for k in range(11+n, 19+n):
    hour_data = prec[k, :, :]
    accumulation += hour_data

    start_hour = dates[k - 1].strftime('%H%M')

    resolution_lon = (lon.max() - lon.min()) / lon.shape[0]
    resolution_lat = (lat.max() - lat.min()) / lat.shape[0]
    transform = from_origin(lon.min(), lat.max(), resolution_lon, resolution_lat)

    tiff_filename_hour = os.path.join(input_hour_dir, f"pch_hour_{start_date}_{start_hour}.tif")

    with rasterio.open(
        tiff_filename_hour,
        'w',
        driver='GTiff',
        height=hour_data.shape[0],
        width=hour_data.shape[1],
        count=1,
        dtype=hour_data.dtype,
        crs=rasterio.crs.CRS.from_proj4("+proj=longlat +datum=WGS84 +no_defs"),
        transform=transform
    ) as dst:
        dst.write(hour_data, 1)

    print(f"Successfully convert 3H netCDF to tiff: {tiff_filename_hour}")

tiff_filename_day = os.path.join(input_day_dir, f"pch_day_{start_date}_2200.tif")

with rasterio.open(
    tiff_filename_day,
    'w',
    driver='GTiff',
    height=accumulation.shape[0],
    width=accumulation.shape[1],
    count=1,
    dtype=accumulation.dtype,
    crs=rasterio.crs.CRS.from_proj4("+proj=longlat +datum=WGS84 +no_defs"),
    transform=transform
) as dst:
    dst.write(accumulation, 1)

print(f"Successfully convert 1D netCDF to tiff: {tiff_filename_day}")

def masked_data(input_tiff, mask_file, output_masked_dir):
    gdf = gpd.read_file(mask_file)
    geometries = [geom for geom in gdf.geometry]

    with rasterio.open(input_tiff) as src:
        out_image, out_transform = mask(src, geometries, crop=True)
        out_meta = src.meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    file_name = os.path.basename(input_tiff)
    output_masked_tiff = os.path.join(output_masked_dir, file_name)

    with rasterio.open(output_masked_tiff, "w", **out_meta) as dst:
        dst.write(out_image)

    print(f"Result masked: {output_masked_tiff}")

for file in os.listdir(input_hour_dir):
    if file.endswith(".tif"):
        input_tiff = os.path.join(input_hour_dir, file)
        masked_data(input_tiff, mask_pulau, output_masked_hour_dir)

for file in os.listdir(input_day_dir):
    if file.endswith(".tif"):
        input_tiff = os.path.join(input_day_dir, file)
        masked_data(input_tiff, mask_pulau, output_masked_day_dir)


Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_0100.tif
Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_0400.tif
Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_0700.tif
Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_1000.tif
Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_1300.tif
Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_1600.tif
Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_1900.tif
Successfully convert 3H netCDF to tiff: ../repository/post-processing/nc_to_tiff/pch_hour_01312025/pch_hour_01312025_2